In [0]:
'''
Descripcion:
    Realiza la depuracion de tablas DELTA por FOLIO finalizado y tablas que tengan mas alla de X dias de creacion
Subetapa:
    MANTENIMIENTO
Trámite:
    MANTENIMIENTO
Tablas INPUT:
    N/A
Tablas OUTPUT:
    N/A
Tablas INPUT DELTA:
    N/A
Tablas OUTPUT DELTA:
    N/A
Archivos SQL:
    MANTENIMIENTO_DELTA_001.sql
'''

In [0]:
%run "./startup"

In [0]:
params = WidgetParams({
    "dias_retencion": str
})

In [0]:
# 📌 Cargar configuraciones globales
# Se establecen variables de configuración necesarias para el proceso
conf = ConfManager()

db = DBXConnectionManager()

query = QueryManager()

In [0]:
df = db.read_data("default", "SELECT DISTINCT SLA.FTC_FOLIO FROM CIERREN.TTCRXGRAL_FOLIO SLA WHERE SLA.FCN_ID_ESTATUS_PROCESO IN ('18') AND FTD_FEH_ACT > SYSDATE - 7")

df.createOrReplaceTempView("MANTEINANCE_FOLIOS_VIEW")

In [0]:
from datetime import datetime, timedelta

dias_retencion = 10 if dbutils.widgets.get("dias_retencion") is None else int(dbutils.widgets.get("dias_retencion"))

logger.info(f"Dias a retener: {dias_retencion}")

# Getting the schema name
catalog_name = spark.sql("show catalogs like 'dbx_mit*'").collect()
spark.sql(f"USE CATALOG {catalog_name[0][0]}")
schema = 'default'
schema_name = catalog_name[0][0] + "." + schema

# List all tables in the schema
tables = spark.sql(f"""SELECT concat_ws('.', table_catalog, table_schema, table_name) as table_name
  FROM {catalog_name[0][0]}.information_schema.tables t
    JOIN MANTEINANCE_FOLIOS_VIEW m
  WHERE table_schema = 'default'
    and table_type = 'MANAGED'
    and lower(table_name) NOT like 'oracle_dispersiones%'
    and table_name like CONCAT('%', m.FTC_FOLIO,'%')
  UNION
  SELECT concat_ws('.', table_catalog, table_schema, table_name) as table_name
  FROM {catalog_name[0][0]}.information_schema.tables t
  WHERE table_schema = 'default'
    and table_type = 'MANAGED'
    and lower(table_name) NOT like 'oracle_dispersiones%'
    and created < current_timestamp() - interval {dias_retencion} day
  """).collect()

for table in tables:
  try:
      spark.sql(f"DROP TABLE {table[0]} PURGE")
      logger.info(f"Dropped table: {table[0]}")
  except Exception as e:
      logger.info(f"No dropped table: {table[0]}")



In [0]:
CleanUpManager.cleanup_notebook(locals())